In [68]:
import spacy
from spacy.lang.en.stop_words import STOP_WORDS
#from keybert import KeyBERT
import yake
import string
import numpy as np

In [2]:
sds_data = pd.read_csv('../data/sds_ds_text.csv')
sds_non_data = pd.read_csv('../data/sds_nds_text.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

## Generating keywords for Data Science Episodes

In [6]:
sds_ds_processed = pd.DataFrame(sds_data.groupby(['episode_number', 'episode_name', 'length_episode', 'context_episode', 'guest_name', 'host_episode', 'episode_year'])['episode_split_text'].agg(lambda x: ' '.join(str(i) for i in x))).reset_index()

<IPython.core.display.Javascript object>

In [7]:
sds_ds_processed

,episode_number,episode_name,length_episode,context_episode,guest_name,host_episode,episode_year,episode_split_text
0,1,"Ruben Kogel on Self-Serve Analytics, R vs Pyt...",42,Business Data Science Database,Ruben Kogel,Kirill Eremenko,2016,This is episode number one with ex-chemical en...
1,2,"Machine Learning, Recommender Systems and the...",51,Machine Learning Data Science,Hadelin de Ponteves,Kirill Eremenko,2016,This is session number two with machine learni...
2,3,"Defining the Data Problem, Data Science in Ma...",53,Machine Learning R Programming Data Science,Dr. Wilson Pok,Kirill Eremenko,2016,This is episode number three with Nanophysics ...
3,4,"Data and Strategy, three Pillars of Research ...",60,Business Data Science,Brendan Hogan,Kirill Eremenko,2016,This is episode four with business strategy ex...
4,5,"Computer Forensics, Fraud Analytics and knowi...",63,Data Science,Dmitry Korneev,Kirill Eremenko,2016,This is episode number five with forensics inv...
...,...,...,...,...,...,...,...,...
446,679,"The A.I. and Machine Learning Landscape, with...",94,Business Data Science Artificial Intelligence,George Mathew,Jon Krohn,2023,This is episode number 679 with George Matthew...
447,680,Automating Industrial Machines with Data Scie...,30,Business Data Science,Allegra Alessi,Jon Krohn,2023,This is episode number 680 with Allegra Alessi...
448,681,XGBoost,72,Machine Learning Data Science Python,Matt Harrison,Jon Krohn,2023,"This is episode number 681 with Matt Harrison,..."
449,682,"Business Intelligence Tools, with Mico Yuk",28,Business Data Science,Mico Yuk,Jon Krohn,2023,"This is episode number 682 with Mico Yuk, host..."


In [10]:
# Remove episode 202 and 546 with no text in episode_split_text
#sds_processed = 
#sds_ds_processed[sds_ds_processed['episode_split_text'] == 'nan']#.reset_index(drop = True)

,episode_number,episode_name,length_episode,context_episode,guest_name,host_episode,episode_year,episode_split_text


In [11]:
# Function to clean the text 
def processed_text(text):  
    # Lowercase the text
    text = text.lower()
    # Remove punctuation from the text
    text = text.translate(str.maketrans("", "", string.punctuation))
    # Remove digits from the text
    text = ''.join(char for char in text if not char.isdigit())
    return text

In [12]:
sds_ds_processed['episode_split_text'] = sds_ds_processed['episode_split_text'].apply(processed_text)

In [14]:
sds_ds_processed['episode_keywords'] = None

In [17]:
# Function to generate 100 keywords per episode using Yake library to store keywords in one column as list
def yake_keyword_extractor(doc):
    global sds_ds_processed
    
    for i, episode_text in doc.iteritems():
        max_ngram_size = 3
        deduplication_threshold = 0.3
        windowSize = 1
        numOfKeywords = 100
        kw_extractor = yake.KeywordExtractor(n = max_ngram_size, dedupLim = deduplication_threshold , windowsSize = windowSize, top = numOfKeywords)
        keywords = kw_extractor.extract_keywords(episode_text)
        sds_ds_processed.at[i, 'episode_keywords'] = keywords

In [18]:
yake_keyword_extractor(sds_ds_processed['episode_split_text'])

In [19]:
sds_ds_processed

,episode_number,episode_name,length_episode,context_episode,guest_name,host_episode,episode_year,episode_split_text,episode_keywords
0,1,"Ruben Kogel on Self-Serve Analytics, R vs Pyt...",42,Business Data Science Database,Ruben Kogel,Kirill Eremenko,2016,this is episode number one with exchemical eng...,"[(data science, 5.201241348387194e-06), (eleme..."
1,2,"Machine Learning, Recommender Systems and the...",51,Machine Learning Data Science,Hadelin de Ponteves,Kirill Eremenko,2016,this is session number two with machine learni...,"[(machine learning machine, 2.506258710542196e..."
2,3,"Defining the Data Problem, Data Science in Ma...",53,Machine Learning R Programming Data Science,Dr. Wilson Pok,Kirill Eremenko,2016,this is episode number three with nanophysics ...,"[(data science podcast, 4.651681915765842e-06)..."
3,4,"Data and Strategy, three Pillars of Research ...",60,Business Data Science,Brendan Hogan,Kirill Eremenko,2016,this is episode four with business strategy ex...,"[(data scientist role, 4.206800003765128e-06),..."
4,5,"Computer Forensics, Fraud Analytics and knowi...",63,Data Science,Dmitry Korneev,Kirill Eremenko,2016,this is episode number five with forensics inv...,"[(unstructured data analytics, 7.2255270954093..."
...,...,...,...,...,...,...,...,...,...
446,679,"The A.I. and Machine Learning Landscape, with...",94,Business Data Science Artificial Intelligence,George Mathew,Jon Krohn,2023,this is episode number with george matthew ma...,"[(product market fit, 2.3243289496297082e-06),..."
447,680,Automating Industrial Machines with Data Scie...,30,Business Data Science,Allegra Alessi,Jon Krohn,2023,this is episode number with allegra alessi io...,"[(product owner role, 6.017532439633584e-06), ..."
448,681,XGBoost,72,Machine Learning Data Science Python,Matt Harrison,Jon Krohn,2023,this is episode number with matt harrison man...,"[(xgboost model yeah, 4.032379247073571e-06), ..."
449,682,"Business Intelligence Tools, with Mico Yuk",28,Business Data Science,Mico Yuk,Jon Krohn,2023,this is episode number with mico yuk host of ...,"[(yeah yeah yeah, 1.3697862978155252e-06), (da..."


In [20]:
def tuple_extractor(row):
    words = [t[0].replace(" ", "_") for t in row]
    return " ".join(words)

In [21]:
sds_ds_processed['keywords'] = sds_ds_processed['episode_keywords'].apply(tuple_extractor)

In [22]:
sds_ds_processed

,episode_number,episode_name,length_episode,context_episode,guest_name,host_episode,episode_year,episode_split_text,episode_keywords,keywords
0,1,"Ruben Kogel on Self-Serve Analytics, R vs Pyt...",42,Business Data Science Database,Ruben Kogel,Kirill Eremenko,2016,this is episode number one with exchemical eng...,"[(data science, 5.201241348387194e-06), (eleme...",data_science elemental_data_problems science_s...
1,2,"Machine Learning, Recommender Systems and the...",51,Machine Learning Data Science,Hadelin de Ponteves,Kirill Eremenko,2016,this is session number two with machine learni...,"[(machine learning machine, 2.506258710542196e...",machine_learning_machine data_data_science lea...
2,3,"Defining the Data Problem, Data Science in Ma...",53,Machine Learning R Programming Data Science,Dr. Wilson Pok,Kirill Eremenko,2016,this is episode number three with nanophysics ...,"[(data science podcast, 4.651681915765842e-06)...",data_science_podcast approach_data_science ’re...
3,4,"Data and Strategy, three Pillars of Research ...",60,Business Data Science,Brendan Hogan,Kirill Eremenko,2016,this is episode four with business strategy ex...,"[(data scientist role, 4.206800003765128e-06),...",data_scientist_role research_youre_working sup...
4,5,"Computer Forensics, Fraud Analytics and knowi...",63,Data Science,Dmitry Korneev,Kirill Eremenko,2016,this is episode number five with forensics inv...,"[(unstructured data analytics, 7.2255270954093...",unstructured_data_analytics data_analytics_tec...
...,...,...,...,...,...,...,...,...,...,...
446,679,"The A.I. and Machine Learning Landscape, with...",94,Business Data Science Artificial Intelligence,George Mathew,Jon Krohn,2023,this is episode number with george matthew ma...,"[(product market fit, 2.3243289496297082e-06),...",product_market_fit large_language_models data_...
447,680,Automating Industrial Machines with Data Scie...,30,Business Data Science,Allegra Alessi,Jon Krohn,2023,this is episode number with allegra alessi io...,"[(product owner role, 6.017532439633584e-06), ...",product_owner_role data_scienceoriented_produc...
448,681,XGBoost,72,Machine Learning Data Science Python,Matt Harrison,Jon Krohn,2023,this is episode number with matt harrison man...,"[(xgboost model yeah, 4.032379247073571e-06), ...",xgboost_model_yeah decision_tree_models book_e...
449,682,"Business Intelligence Tools, with Mico Yuk",28,Business Data Science,Mico Yuk,Jon Krohn,2023,this is episode number with mico yuk host of ...,"[(yeah yeah yeah, 1.3697862978155252e-06), (da...",yeah_yeah_yeah data_storytelling_framework ana...


In [23]:
sds_ds_processed.to_csv('../data/sds_ds_keywords.csv', index = False)

## Generating keywords for non-Data Science Episodes

In [24]:
sds_nds_processed = pd.DataFrame(sds_non_data.groupby(['episode_number', 'episode_name', 'length_episode', 'context_episode', 'guest_name', 'host_episode', 'episode_year'])['episode_split_text'].agg(lambda x: ' '.join(str(i) for i in x))).reset_index()

<IPython.core.display.Javascript object>

In [26]:
sds_nds_processed['episode_split_text'] = sds_nds_processed['episode_split_text'].apply(processed_text)

In [30]:
sds_nds_processed['episode_keywords'] = None

In [31]:
# Function to generate 100 keywords per episode using Yake library to store keywords in one column as list
def yake_keyword_extractor(doc):
    global sds_nds_processed
    
    for i, episode_text in doc.iteritems():
        max_ngram_size = 3
        deduplication_threshold = 0.3
        windowSize = 1
        numOfKeywords = 100
        kw_extractor = yake.KeywordExtractor(n = max_ngram_size, dedupLim = deduplication_threshold , windowsSize = windowSize, top = numOfKeywords)
        keywords = kw_extractor.extract_keywords(episode_text)
        sds_nds_processed.at[i, 'episode_keywords'] = keywords

In [32]:
yake_keyword_extractor(sds_nds_processed['episode_split_text'])

In [33]:
sds_nds_processed['keywords'] = sds_nds_processed['episode_keywords'].apply(tuple_extractor)

In [34]:
sds_nds_processed

,episode_number,episode_name,length_episode,context_episode,guest_name,host_episode,episode_year,episode_split_text,episode_keywords,keywords
0,22,SEOI PEI – The skin of water listen now,10,Data Science,Kirill Eremenko,Kirill Eremenko,2017,this is episode number and today were kicking...,"[(minute friday sessionshello, 9.7043007419160...",minute_friday_sessionshello thousand_people_li...
1,24,“What are you passionate about?”,5,Data Science,Kirill Eremenko,Kirill Eremenko,2017,this is five minute friday episode number and...,"[(theyre passionate completely, 9.532967431373...",theyre_passionate_completely fantastic_time_co...
2,26,Sweat every day,6,Data Science,Kirill Eremenko,Kirill Eremenko,2017,this is five minute friday episode number swe...,"[(body temperature meaning, 9.127388236232729e...",body_temperature_meaning sweating_releases_end...
3,27,"Career choice, disruptions in finance and app...",62,Uncategorized,Aziz Mamatov,Kirill Eremenko,2017,this is episode number with vice president of...,"[(data science yeah, 3.2346981864497386e-06), ...",data_science_yeah finance_world_interesting ye...
4,28,Pride and humility,5,Data Science,Kirill Eremenko,Kirill Eremenko,2017,this is five minute friday episode number pri...,"[(great accomplishment great, 0.00017971430030...",great_accomplishment_great project_youre_delay...
...,...,...,...,...,...,...,...,...,...,...
226,616,The Four Requirements for Expertise (beyond t...,6,Data Science Career Tips,Jon Krohn,Jon Krohn,2022,this is fiveminute friday on the requirements ...,"[(chess piano tennis, 0.00012765156512494006),...",chess_piano_tennis repeated_timely_feedback cl...
227,618,The Joy of Atelic Activities,4,Life Philosophy Data Science,Jon Krohn,Jon Krohn,2022,this is fiveminute friday on the joy of atelic...,"[(atelic activities include, 0.000101707939323...",atelic_activities_include enriching_life_exper...
228,622,Burnout,24,Business Data Science Productivity Career Tips,Christina Maslach,Jon Krohn,2022,this is episode number with professor christi...,"[(chronic job stressors, 2.249540632751539e-05...",chronic_job_stressors workload_theyre_talking ...
229,640,What I Learned in 2022,37,Data Science,Jon Krohn,Jon Krohn,2022,this is fiveminute friday on what i learned in...,"[(year episode number, 3.7707061236867265e-06)...",year_episode_number machine_learning_company y...


In [35]:
sds_nds_processed.to_csv('../data/sds_nds_keywords.csv', index = False)

### Count Vectorizer

In [36]:
from sklearn.feature_extraction.text import CountVectorizer
import numpy as np

In [37]:
sds_ds_keywords = pd.read_csv('../data/sds_ds_keywords.csv')

<IPython.core.display.Javascript object>

In [39]:
sds_ds_keywords['keywords'].isna().value_counts()

False    451
Name: keywords, dtype: int64

In [45]:
# Create a Vectorizer Object
vect= CountVectorizer()

In [56]:
vector = vect.fit_transform(sds_ds_keywords['keywords'])

In [47]:
vect.vocabulary_

{'data_science': 6367,
 'elemental_data_problems': 8107,
 'science_skill_set': 23655,
 't_wanna_work': 26654,
 'problem_solving_thing': 21136,
 'handle_health_data': 12023,
 'business_customers_people': 3474,
 're_technical_customers': 22287,
 'analytics_team_grows': 1021,
 'algorithm_people_specialize': 700,
 'build_scalable_analytics': 3358,
 'chemical_engineering_background': 4294,
 'red_shift_inaudible': 22565,
 'offline_analysis_building': 18998,
 're_ninetyfive_percent': 22258,
 'mentioned_communication_skills': 17511,
 'great_day_wonderful': 11589,
 'interesting_project_awesome': 14276,
 'found_pretty_interesting': 10200,
 'experiments_quickly_build': 9045,
 'wagon_wagon_yeah': 29589,
 'yeah_totally_surprisingly': 30675,
 'building_offline_model': 3400,
 'insights_beautiful_love': 14035,
 'company_called_wagon': 4990,
 'measuring_student_satisfaction': 17377,
 'tables_dashboards_web': 26702,
 'started_exploring_red': 25525,
 'bigger_longer_term': 2617,
 'wizard_ruben_kogelwelcom

In [ ]:
# Encode the Document
#count_vector = vect.transform(sds_processed['keywords'])

In [49]:
sum(vect.vocabulary_.values()) 

485301435

In [51]:
len(vect.vocabulary_)

31155

In [57]:
vector.shape

(451, 31155)

In [67]:
word_counts = pd.DataFrame({
    'words': vect.vocabulary_.keys(), 
    'frequency': vect.vocabulary_.values()
})

word_counts.head()

<IPython.core.display.Javascript object>

,words,frequency
0,data_science,6367
1,elemental_data_problems,8107
2,science_skill_set,23655
3,t_wanna_work,26654
4,problem_solving_thing,21136


In [73]:
word_counts = pd.DataFrame({
    'words': vect.get_feature_names_out(),
    'frequency': np.array(vector.sum(axis = 0)).flatten()
})
word_counts.sort_values('frequency', ascending = False)

<IPython.core.display.Javascript object>

,words,frequency
6249,data,368
30548,yeah,254
16380,lot,193
27743,time,189
24177,show,176
...,...,...
10769,gdpr_legal_services,1
10768,gdpr_global,1
10767,gdpr,1
10765,gdansk_university,1


In [74]:
word_counts[word_counts['frequency'] < 3]

,words,frequency
0,_affairs,1
1,_capacity_enabling,1
2,_digital,1
3,_guys,1
4,_hands,1
...,...,...
31150,zuckerberg_gray,1
31151,zuckerbergthe_fourth_mentor,1
31152,zurich,1
31153,zurich_in_switzerland,1


In [78]:
word_counts.sort_values('frequency', ascending=False).head(150)

,words,frequency
6249,data,368
30548,yeah,254
16380,lot,193
27743,time,189
24177,show,176
...,...,...
10038,focus,15
26487,system,15
24355,similar,15
20281,play,15


In [ ]:
word_counts.sort_values('frequency', ascending = False)

In [ ]:
sds_processed

In [ ]:
# Function to generate 100 keywords per episode using Yake library if keywords need to be separate columns

def yake_keyword_extractor(doc):
    global sds_processed

    for i, episode_text in doc.iteritems():
        max_ngram_size = 3
        deduplication_threshold = 0.3
        windowSize = 1
        numOfKeywords = 100
        kw_extractor = yake.KeywordExtractor(n=max_ngram_size, dedupLim=deduplication_threshold, windowsSize=windowSize, top=numOfKeywords)
        keywords = kw_extractor.extract_keywords(episode_text)

        # Create separate columns for each keyword
        for j, keyword in enumerate(keywords):
            column_name = f'episode_keyword{j+1}'
            sds_processed.at[i, column_name] = keyword[0]  # Store the keyword value in the respective column

In [ ]:

 
# Create a Vectorizer Object
vectorizer = CountVectorizer()
 
vectorizer.fit(document)
 
# Printing the identified Unique words along with their indices
print("Vocabulary: ", vectorizer.vocabulary_)
 
# Encode the Document
vector = vectorizer.transform(document)
 
# Summarizing the Encoded Texts
print("Encoded Document is:")
print(vector.toarray())

In [ ]:
sds_1.shape

In [ ]:
sds_processed

In [ ]:
sds_processed['episode_keywords'].loc[20]

In [ ]:
sds_processed.info()

In [ ]:
#sds_processed.to_csv('../data/sds_yake_keywords.csv')

In [ ]:
#sds_keywords = pd.read_csv('../data/sds_yake_keywords.csv')

In [ ]:
sds_1 = sds_processed[sds_processed['episode_number'] == 1]

In [ ]:
sds_1

In [ ]:
sds_1.info()

In [ ]:
sds_1['episode_keywords'].loc[0]

In [ ]:
sds_1

In [ ]:
sds_1['keywords'] = None

In [ ]:
sds_processed['keywords'] = None

In [ ]:
print(sds_processed['keywords'].loc[677])

In [ ]:
sds_1['episode_keywords'].apply(lambda x: tuple_extractor(x))

In [ ]:
sds_1.loc[0]

In [ ]:
sds_1['keywords'].loc[0]

In [ ]:
sds_1['keywords'].loc[0]

In [ ]:
def tuple_extractor(word_tuples):
    global sds_1
    words = [t[0] for t in word_tuples]
    return words

In [ ]:
sds_1['keywords'] = sds_1['episode_keywords'].apply(lambda x: tuple_extractor(x))

In [ ]:
sds_1

In [ ]:
sds_1['keywords'] = sds_1['episode_keywords'].apply(tuple_extractor)

In [ ]:
sds_1['keywords'].loc[0]

In [ ]:
def tuple_extractor(tuple):
    global sds_keywords
    words = []
    #for i, episode_text in doc.iteritems():
    for i, word_tuple in tuple.iteritems():
        words = [t[0] for t in word_tuple]
        sds_keywords['keywords'] = words
    
            

In [ ]:
sds_keywords['episode_keywords'].apply(tuple_extractor)

In [ ]:
sds_keywords

In [ ]:
sds_1['keywords'] = sds_1['episode_keywords'].apply(tuple_extractor)

In [ ]:
sds_1

In [ ]:
[t[0] for t in sds_1['episode_keywords']]

In [ ]:
sds_processed['episode_keywords'] = [t[0] for t in sds_processed['episode_keywords']]

In [ ]:
sds_processed

In [ ]:
sds_processed['episode_keywords'].loc[678]

In [ ]:
def keyword_extractor(doc):
    kw_model = KeyBERT()
    global sds_processed
    stopwords = list(STOP_WORDS)
    
    for i, episode_text in doc.iteritems():
        keywords = kw_model.extract_keywords(episode_text, keyphrase_ngram_range=(1, 1), stop_words = stopwords, top_n = 100, use_mmr=True, diversity=0.7)
        sds_processed.at[i, 'episode_keywords'] = keywords

In [ ]:
kw_model = KeyBERT()
stopwords = list(STOP_WORDS)
keywords = kw_model.extract_keywords(doca, keyphrase_ngram_range=(1, 2), stop_words = stopwords, top_n = 100, use_mmr=True, diversity=0.8)



In [ ]:
keywords

In [ ]:
keyword_extractor(sds_processed['episode_split_text'])

In [ ]:
len(keywords)

In [ ]:
keywords

In [ ]:
#sds_processed['episode_split_text'][1:5].apply(keyword_extractor)

In [ ]:
# https://maartengr.github.io/BERTopic/index.html